In [ ]:
# Install required libraries
!pip install transformers datasets scikit-learn torch --quiet

In [ ]:
pip install --upgrade transformers datasets --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.3/506.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 23.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.


In [ ]:
!pip uninstall -y transformers datasets pyarrow



Found existing installation: transformers 4.57.1
Uninstalling transformers-4.57.1:
  Successfully uninstalled transformers-4.57.1
Found existing installation: datasets 4.2.0
Uninstalling datasets-4.2.0:
  Successfully uninstalled datasets-4.2.0
Found existing installation: pyarrow 21.0.0
Uninstalling pyarrow-21.0.0:
  Successfully uninstalled pyarrow-21.0.0


In [ ]:

!pip install -U transformers datasets scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.4 MB/s eta 0:00:00
  Using cached datasets-4.2.0-py3-none-any.whl.metadata (18 kB)
  Using cached pyarrow-21.0.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 88.5 MB/s eta 0:00:00
Using cached datasets-4.2.0-py3-none-any.whl (506 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 66.9 MB/s eta 0:00:00
Using cached pyarrow-21.0.0-cp312-cp312-manylinux_2_28_x86_64.whl (42.8 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompat

In [ ]:
# -------------------------------
# Step 1: Imports
# -------------------------------
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import re

# -------------------------------
# Step 2: Load dataset
# -------------------------------
df = pd.read_csv("airline_customer_requests.csv")  # replace with your CSV file
print("Dataset size:", len(df))
print(df.head())

# -------------------------------
# Step 3: Preprocessing function
# -------------------------------
def preprocess_text(text):
    text = str(text).lower()  # lowercase
    text = re.sub(r"\s+", " ", text)  # remove extra spaces
    text = re.sub(r"[^a-z0-9\s?.!,]", "", text)  # remove unwanted chars
    text = text.strip()
    return text

df['utterance'] = df['utterance'].apply(preprocess_text)

# -------------------------------
# Step 4: Encode labels
# -------------------------------
labels = df['intent'].unique().tolist()
label2id = {label: idx for idx, label in enumerate(labels)}
id2label = {idx: label for label, idx in label2id.items()}
df['label'] = df['intent'].map(label2id)

# -------------------------------
# Step 5: Split dataset
# -------------------------------
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['utterance'], df['label'], test_size=0.1, random_state=42, stratify=df['label']
)

# -------------------------------
# Step 6: Tokenization
# -------------------------------
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

def tokenize(batch):
    return tokenizer(batch['utterance'], padding='max_length', truncation=True, max_length=128)

train_dataset = Dataset.from_dict({'utterance': train_texts.tolist(), 'label': train_labels.tolist()})
test_dataset = Dataset.from_dict({'utterance': test_texts.tolist(), 'label': test_labels.tolist()})

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

# -------------------------------
# Step 7: Load model
# -------------------------------
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id
)

# -------------------------------
# Step 8: Training arguments
# -------------------------------
training_args = TrainingArguments(
    output_dir='./distilbert_airline',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir='./logs',
    logging_steps=100,
    save_total_limit=1,
    learning_rate=2e-5
)

# -------------------------------
# Step 9: Metrics
# -------------------------------
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'precision': precision, 'recall': recall, 'f1': f1}

# -------------------------------
# Step 10: Trainer
# -------------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# -------------------------------
# Step 11: Train
# -------------------------------
trainer.train()

# -------------------------------
# Step 12: Evaluate
# -------------------------------
results = trainer.evaluate()
print("✅ Evaluation Results:")
for k, v in results.items():
    print(f"{k}: {v:.4f}")

# -------------------------------
# Step 13: Save model
# -------------------------------
model.save_pretrained('./distilbert_airline')
tokenizer.save_pretrained('./distilbert_airline')
print("✅ Model and tokenizer saved successfully!")


Dataset size: 31024
                                      utterance                intent
0                  Are knives allowed on board?  Prohibited Items Faq
1                  I need wheelchair assistance    Special Assistance
2      Does the insurance cover booking delays?             Insurance
3  Assistance at airport for elderly passengers    Special Assistance
4                Is ticket insurance included??             Insurance


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 22i261 (22i261-psg-college-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
100,2.908600
200,1.521600
300,0.513500
400,0.134500
500,0.049700
600,0.029300
700,0.020300
800,0.014600
900,0.011300
1000,0.009200


✅ Evaluation Results:
eval_loss: 0.0003
eval_accuracy: 1.0000
eval_precision: 1.0000
eval_recall: 1.0000
eval_f1: 1.0000
eval_runtime: 11.2674
eval_samples_per_second: 275.3960
eval_steps_per_second: 17.2180
epoch: 3.0000
✅ Model and tokenizer saved successfully!


In [9]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
import torch

# Load saved model & tokenizer
model_path = './distilbert_airline'
tokenizer = DistilBertTokenizerFast.from_pretrained(model_path)
model = DistilBertForSequenceClassification.from_pretrained(model_path)
model.eval()

# Example text
text = ""

# Tokenize
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)

# Predict
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_id = torch.argmax(logits, dim=1).item()

# Get label using id2label from model config
predicted_label = model.config.id2label[predicted_class_id]

print("Predicted intent:", predicted_label)


Predicted intent: Change Flight


In [ ]:
from google.colab import drive
drive.mount('/content/drive')